In [61]:
from openai import OpenAI
from credentials import api_key

In [62]:
client = OpenAI(api_key=api_key)

In [63]:
batch_input_file = client.files.create(
    file=open("output.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file)

FileObject(id='file-39hTsg2D1ZAH8LQX9mtEhq', bytes=1458059, created_at=1735754251, filename='output.jsonl', object='file', purpose='batch', status='processed', status_details=None)


In [64]:
batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "nightly eval job"
    },
    
)

Batch(id='batch_6775820cd8c081908ca349e8a463fc4a', completion_window='24h', created_at=1735754252, endpoint='/v1/chat/completions', input_file_id='file-39hTsg2D1ZAH8LQX9mtEhq', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1735840652, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [83]:
batch = client.batches.retrieve("batch_6775820cd8c081908ca349e8a463fc4a")
print(batch)

Batch(id='batch_6775820cd8c081908ca349e8a463fc4a', completion_window='24h', created_at=1735754252, endpoint='/v1/chat/completions', input_file_id='file-39hTsg2D1ZAH8LQX9mtEhq', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1735840652, failed_at=None, finalizing_at=None, in_progress_at=1735754253, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=3, total=8))


In [77]:
file_response = client.files.content("file-39hTsg2D1ZAH8LQX9mtEhq")
print(file_response.text)

{"custom_id": "file_1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-mini", "messages": [{"role": "system", "content": "Context: We are social science researchers studying how Roe v. Wade has been cited across different courts, times, and jurisdictions. We seek detailed yet consistent information from each case.\nFormat Requirements:\nAnswer each question in a new line under the corresponding number (or letter), without repeating the question.\nFor yes/no questions, answer only \"yes\" or \"no\" (all lowercase). If you cannot determine an answer, write \"No information found.\"\nWhen providing quotes/excerpts, limit them to 3\u00e2\u20ac\u201c5 lines each.\nIf the requested information is not found or not applicable, write \"No information found.\"\nLevel of Detail:\nInclude exact names, excerpts from the decision, and indicate whether statements appear in the majority, dissenting, or concurring opinions (if applicable).\nModel Note:\nApply advanced reason